In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import pandas as pd
import pickle
import random
from kcs_ml_infr import *

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model

from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier

# Make annoying KNN warning go away since I'm not going to edit scikit learn's code lol
import warnings
warnings.filterwarnings("ignore")

random.seed(a=1)

In [2]:
path = r'C:\Users\kdmen\Desktop\Research\Data'

#with open(path + r'\cphs_data_block1.pickle', 'rb') as handle:
with open(path + r'\continuous_full_data_block1.pickle', 'rb') as handle:
    #refs_block1, poss_block1, dec_vels_block1, int_vel_block1, emgs_block1, Ws_block1, Hs_block1, alphas_block1, pDs_block1, times_block1, conditions_block1 = pickle.load(handle)
    _, _, _, _, _, Ws_block1, _, _, _, _, _ = pickle.load(handle)

#with open(path + r'\cphs_data_block2.pickle', 'rb') as handle:
with open(path + r'\continuous_full_data_block2.pickle', 'rb') as handle:
    #refs_block2, poss_block2, dec_vels_block2, int_vel_block2, emgs_block2, Ws_block2, Hs_block2, alphas_block2, pDs_block2, times_block2, conditions_block2 = pickle.load(handle)
    _, _, _, _, _, Ws_block2, _, _, _, _, _ = pickle.load(handle)
    
update_ix = np.load(path+r"\update_ix.npy")
print(update_ix.shape)
print(update_ix)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\kdmen\\Desktop\\Research\\Data\\continuous_full_data_block1.pickle'

In [ ]:
key_to_num = dict()
num_to_key = dict()
for idx, key in enumerate(keys):
    key_to_num[key] = idx
    num_to_key[idx] = key
    
cv = 5
my_metrics_cols = ['Algorithm', 'One Off Acc', 'CV Acc', 'K Folds']
my_models = [LogisticRegression(), KNeighborsClassifier(), LinearSVC(), DecisionTreeClassifier(), GradientBoostingClassifier()]

In [ ]:
keys = keys[:7]

In [ ]:
num_conds = 8
dec_flattened_df1 = pd.DataFrame(columns=["Subject", "Condition", "Update Number", "Flattened Decoder"])
dec_flattened_df2 = pd.DataFrame(columns=["Subject", "Condition", "Update Number", "Flattened Decoder"])

for key in keys:
    participant_dec1 = Ws_block1[key]
    participant_dec2 = Ws_block2[key]
    
    for my_cond in range(num_conds):
        for update_number, update_idx in enumerate(update_ix):
            dec_flattened_df1.loc[len(dec_flattened_df1)] = [key, my_cond, update_number, np.ravel(participant_dec1[my_cond, update_idx, :, :])]
            dec_flattened_df2.loc[len(dec_flattened_df2)] = [key, my_cond, update_number, np.ravel(participant_dec2[my_cond, update_idx, :, :])]
        
dec_flattened_df = pd.concat((dec_flattened_df1, dec_flattened_df2))

print(dec_flattened_df.shape)
dec_flattened_df.head()

In [ ]:
flat_dec_expanded_df = pd.DataFrame()
for my_row in range(dec_flattened_df.shape[0]):
    test=pd.DataFrame(dec_flattened_df.iloc[my_row,3]).T
    flat_dec_expanded_df = pd.concat((flat_dec_expanded_df, test))

flat_dec_expanded_df.reset_index(inplace=True, drop=True)
flat_dec_expanded_df.insert(loc=0, column='Update Number', value=list(dec_flattened_df['Update Number']))
flat_dec_expanded_df.insert(loc=0, column='Condition', value=list(dec_flattened_df['Condition']))
flat_dec_expanded_df.insert(loc=0, column='Subject', value=list(dec_flattened_df['Subject']))

print(flat_dec_expanded_df.shape)
flat_dec_expanded_df.head()

In [ ]:
flat_dec_expanded_df.columns = flat_dec_expanded_df.columns.astype(str)

In [ ]:
zero_df, zero_test_df = nth_decoder_model(flat_dec_expanded_df, 0, my_models, test=True)
zero_test_df.head(10)

In [ ]:
my_metrics_cols=['Algorithm', 'One Off Acc', 'CV Acc', 'K Folds']
key_to_num_dict=key_to_num

In [ ]:
n=0
nth_update_idxs = flat_dec_expanded_df[~(flat_dec_expanded_df['Update Number'] == n)].index
dec0_df = flat_dec_expanded_df.drop(nth_update_idxs)
dec0_df.head(8)

In [ ]:
n=1
nth_update_idxs = flat_dec_expanded_df[~(flat_dec_expanded_df['Update Number'] == n)].index
dec1_df = flat_dec_expanded_df.drop(nth_update_idxs)
dec1_df.head(8)

In [ ]:
n=2
nth_update_idxs = flat_dec_expanded_df[~(flat_dec_expanded_df['Update Number'] == n)].index
dec2_df = flat_dec_expanded_df.drop(nth_update_idxs)
dec2_df.head(8)

In [ ]:
zero_df, zero_test_df = nth_decoder_model(flat_dec_expanded_df, 0, my_models, test=True)
zero_test_df.head(10)

In [ ]:
print(1)
one_df, one_test_df = nth_decoder_model(flat_dec_expanded_df, 1, my_models, test=True)
two_df, two_test_df = nth_decoder_model(flat_dec_expanded_df, 2, my_models, test=True)
three_df, three_test_df = nth_decoder_model(flat_dec_expanded_df, 3, my_models, test=True)
four_df, four_test_df = nth_decoder_model(flat_dec_expanded_df, 4, my_models, test=True)
five_df, five_test_df = nth_decoder_model(flat_dec_expanded_df, 5, my_models, test=True)
six_df, six_test_df = nth_decoder_model(flat_dec_expanded_df, 6, my_models, test=True)
print(7)
seven_df, seven_test_df = nth_decoder_model(flat_dec_expanded_df, 7, my_models, test=True)
eight_df, eight_test_df = nth_decoder_model(flat_dec_expanded_df, 8, my_models, test=True)
nine_df, nine_test_df = nth_decoder_model(flat_dec_expanded_df, 9, my_models, test=True)
ten_df, ten_test_df = nth_decoder_model(flat_dec_expanded_df, 10, my_models, test=True)
ele_df, ele_test_df = nth_decoder_model(flat_dec_expanded_df, 11, my_models, test=True)
print(12)
twe_df, twe_test_df = nth_decoder_model(flat_dec_expanded_df, 12, my_models, test=True)
thirt_df, thirt_test_df = nth_decoder_model(flat_dec_expanded_df, 13, my_models, test=True)
frtn_df, frtn_test_df = nth_decoder_model(flat_dec_expanded_df, 14, my_models, test=True)
fftn_df, fftn_test_df = nth_decoder_model(flat_dec_expanded_df, 15, my_models, test=True)
sixtn_df, sixtn_test_df = nth_decoder_model(flat_dec_expanded_df, 16, my_models, test=True)
svntn_df, svntn_test_df = nth_decoder_model(flat_dec_expanded_df, 17, my_models, test=True)
print('Complete')

In [ ]:
eightn_df, eightn_test_df = nth_decoder_model(flat_dec_expanded_df, 18, my_models, test=True)
eightn_test_df.head(10)

In [ ]:
full_test_df = pd.concat((zero_test_df, one_test_df, two_test_df, three_test_df, four_test_df, 
           five_test_df, six_test_df, seven_test_df, eight_test_df, nine_test_df, 
           ten_test_df, ele_test_df, twe_test_df, thirt_test_df, frtn_test_df, 
           fftn_test_df, sixtn_test_df, svntn_test_df, eightn_test_df))

full_test_df.drop('K Folds', axis=1, inplace=True)
print(full_test_df.shape)

In [ ]:
lr_df = full_test_df[full_test_df['Algorithm']=='LogisticRegression()']
knn_df = full_test_df[full_test_df['Algorithm']=='KNeighborsClassifier()']
svc_df = full_test_df[full_test_df['Algorithm']=='LinearSVC()']
dt_df = full_test_df[full_test_df['Algorithm']=='DecisionTreeClassifier()']
gbt_df = full_test_df[full_test_df['Algorithm']=='GradientBoostingClassifier()']

In [ ]:
plt.figure(figsize=(9,7))
plt.plot(list(range(19)), lr_df.iloc[:, 2], label='LogReg')
plt.plot(list(range(19)), knn_df.iloc[:, 2], label='KNN')
plt.plot(list(range(19)), svc_df.iloc[:, 2], label='SVC')
plt.plot(list(range(19)), dt_df.iloc[:, 2], label='DTree')
plt.plot(list(range(19)), gbt_df.iloc[:, 2], label='GBC')
plt.xticks(np.arange(0, 19, 1.0))
plt.yticks(np.arange(0, 120, 20.0))
plt.grid(axis='y')
plt.xlabel('Update Number')
plt.ylabel('Testing Accuracy')
plt.title('Model Accuracy as a function of Decoder Update Number')
plt.legend(loc='best')
plt.show()

In [ ]:
my_x_updates = list(range(19))[2:]

plt.figure(figsize=(9,7))
plt.plot(my_x_updates, lr_df.iloc[2:, 2], label='LogReg')
plt.plot(my_x_updates, knn_df.iloc[2:, 2], label='KNN')
plt.plot(my_x_updates, svc_df.iloc[2:, 2], label='SVC')
plt.plot(my_x_updates, dt_df.iloc[2:, 2], label='DTree')
plt.plot(my_x_updates, gbt_df.iloc[2:, 2], label='GBC')
plt.xticks(np.arange(2, 19, 1.0))
plt.yticks(np.arange(0, 120, 20.0))
plt.grid(axis='y')
plt.xlabel('Update Number')
plt.ylabel('Testing Accuracy')
plt.title('Model Accuracy as a function of Decoder Update Number')
plt.legend(loc='best')
plt.show()

In [ ]:
assert(1==0)

In [ ]:
# Save model dfs so I don't have to rerun all this code
lr_df.to_pickle(r"Saved_Metrics\104P_full_lr_df.pkl")
knn_df.to_pickle(r"Saved_Metrics\104P_full_knn_df.pkl")
svc_df.to_pickle(r"Saved_Metrics\104P_full_svc_df.pkl")
dt_df.to_pickle(r"Saved_Metrics\104P_full_dt_df.pkl")
gbt_df.to_pickle(r"Saved_Metrics\104P_full_gbt_df.pkl")